In [14]:
# pip freeze > requirements.txt

In [8]:
import ray, os
ray.init(num_cpus=32)
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as md

2022-11-14 06:23:11,763	INFO worker.py:1528 -- Started a local Ray instance.


In [33]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from dotenv import load_dotenv
load_dotenv('env')

True

In [9]:
business = md.read_csv('http://s3fld.s3.us-west-1.amazonaws.com/599Team12/business.csv')
review = md.read_csv('http://s3fld.s3.us-west-1.amazonaws.com/599Team12/review.csv')

2022-11-14 06:23:17,758	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.


In [10]:
resturant_business = business[business['categories'].str.lower().str.contains('restaurants', na=False)]
resturant_business = resturant_business[['business_id','name','address','city','state','postal_code','latitude','longitude','stars']]
resturant_business.sample(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars
64863,yjYOdci9mTeZphr-WD3H9Q,Lil' Java Philly,1739 W Ritner St,Philadelphia,PA,19145,39.921534,-75.176775,4.5
73545,S-gk6QdFM08F8HlJaUmftw,Nudy's Café - Paoli,31 W Lancaster Ave,Paoli,PA,19301,40.042057,-75.485414,3.0
64415,z-2pRvQXAWqcZdLW2fKj9g,Deerhead Hot Dogs,620 S Maryland Ave,Wilmington,DE,19804,39.723895,-75.592028,3.5
14074,JaHuWM3Tp1aG7NW66mhutw,McDonald's,3014 New Castle Ave,New Castle,DE,19720,39.700996,-75.551487,2.0
30394,xoSZvVW1isvWSGXkLaV7dg,Trailers A Go Go,"535 Marlborough St, Unit C",Oldsmar,FL,34677,28.038868,-82.671993,4.0


In [11]:
resturant_review = review[review['business_id'].isin(resturant_business['business_id'])]
resturant_review['date'] = pd.to_datetime(resturant_review['date'], format='%Y-%m-%d')
resturant_review = resturant_review[['review_id', 'business_id','stars','text','date']]
resturant_review.sample(5)

(apply_list_of_funcs pid=2124) 2022-11-14 06:25:26,946	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=2129) 2022-11-14 06:25:26,967	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=2132) 2022-11-14 06:25:26,967	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=2133) 2022-11-14 06:25:26,967	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.
(apply_list_of_funcs pid=2128) 2022-11-14 06:25:26,957	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a 

,review_id,business_id,stars,text,date
3157235,I8hcjAceO1N6WuWa9u2A0Q,Z_WZq3uFGqvtjsFl6kNBQA,5,Went down to Campo for lunch on a Thursday aro...,2012-10-06 23:58:54
622437,7V2fQtW101Pp-dDCURYJQQ,eaDZlSuVS0EY67Ke6pRP6Q,2,I've here many times ago and the food was fine...,2018-07-12 02:05:15
1546157,GDTiVyNwcIaV9Z-yLLoWvA,pnZQOGEV-Ru6vHIYaUuQBw,4,Very fast service! We had the spring rolls w/p...,2017-09-13 04:33:11
4652235,WNTxOl9T8vIeh5vpgsuIuQ,a_oRfgkNFnulTada--P_Tg,4,Came across SBC online and it seems to have po...,2020-01-24 17:47:31
25327,BTGbizHFsF1HYjkifwjXgw,zVlhs7BVaouutt-rles-YA,5,Once you eat here you will regret ever having ...,2014-08-27 20:05:47


In [12]:
resturant_name_dict = resturant_business.set_index('business_id')['name'].to_dict()
resturant_review['resturant_name'] = resturant_review['business_id'].map(resturant_name_dict)
resturant_review.sample(5)

,review_id,business_id,stars,text,date,resturant_name
926241,dzZy0zUfzwUYEJQjsmshcg,z3TB52s5p5PcRsw2T3J6MQ,2,This place is over rated and charged for every...,2018-06-15 01:54:06,Chicken and Taco Loco
1690545,RSjw8ETHKxow8YWLcP6FFQ,U3grYFIeu6RgAAQgdriHww,5,"This place was phenomenal. Coming from Boston,...",2015-10-03 03:29:40,Brophy Bros - Santa Barbara
6420406,sGpD0wCJMJV7MTmAzpLYrA,3CGXa85uNgvp8Berx5AhQQ,4,The main courses are excellent. I've tried th...,2014-09-14 22:29:03,La Margarita
5086014,RBh8gjvFulFiInbV7ya8YA,cQIh4YJlVtZI9TLF5_smOg,4,"So, I arrived a little tipsy at the Shaved Duc...",2011-02-23 23:35:33,The Shaved Duck
5241203,juhAHcBE-nDnft5REaY17Q,OxQK9EMUXqynT7dbWAP9wA,5,"Fantastic BBQ joint!! Great atmosphere, tende...",2019-05-26 20:25:01,Mission BBQ


In [13]:
resturant_review['sentiment'] = resturant_review['stars'].apply(lambda x: 'negative' if x <= 2 else ('positive' if x >= 4 else 'neutral'))
resturant_review.sample(5)

,review_id,business_id,stars,text,date,resturant_name,sentiment
3060058,qsUU2lX_Sf5cyfhoXqJCRg,fOhnSqmO4XY5vSI8whVKSA,5,"You can do a really good breakfast, brunck or ...",2014-02-03 22:59:37,Farmicia,positive
5056044,JdAXKEbW5YCmQ45LT8pnsw,bRlpi6k7wTSHHEapcLz_KA,1,Okay I love chipotle and it hurts to give such...,2016-04-28 20:59:58,Chipotle Mexican Grill,negative
3452284,yYvl2KcvW84mNzJQbOlb3g,XzzGAAk_E3FJ9bdri71oCA,3,Maybe I caught them on an off day. Got a phil...,2020-09-22 03:00:09,IT'Z A Philly Thing,neutral
393660,-OoSrUiiqMR9kbSaIQQb5w,d_tRshM-w6S4QxE4VVi8tQ,5,Hands down BEST BRUNCH IN PHILLY! All you can ...,2014-08-07 01:47:01,Jones,positive
5427968,SEDaW50s1TSb2a_0YHE1cg,ZAj8Zkl5kjDOZTaTNR1Zdw,5,Out for a little shopping Love their hawian pi...,2013-12-09 00:13:34,DeLosa's Pizza,positive


In [16]:
resturant_business.to_csv('data/business.csv', compression = 'zip')
resturant_review.to_csv('data/review.csv', compression = 'zip')

In [39]:
s3_client.upload_file('data/business.csv', 's3fld', '599Team12/resturants/business.csv')
s3_client.upload_file('data/review.csv', 's3fld', '599Team12/resturants/review.csv')

In [22]:
# !pip freeze > requirements.txt